In [116]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [117]:
df = pd.read_csv('Nigeria House.csv')

In [118]:
df.head()

,bedrooms,bathrooms,toilets,parking spaces,type,town,state,price
0,5.0,6.0,6.0,3.0,block of flats,Dakibiyu,Abuja,49000000
1,5.0,5.0,6.0,3.0,detached duplex,Lekki,Lagos,150000000
2,5.0,5.0,6.0,4.0,detached duplex,Lekki,Lagos,120000000
3,4.0,5.0,5.0,3.0,detached duplex,Ajah,Lagos,150000000
4,NaN,NaN,2.0,NaN,house,Lekki,Lagos,25000000


In [119]:
df['price'] = df['price'].str.replace(r'(\n\$\d+ \n[a-z]+\. ₦|\n)','',regex =True).astype(np.float)

In [120]:
df = df.drop(df[df['type'] == 'house'].index)   

In [121]:
df.shape

(35554, 8)

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35554 entries, 0 to 37981
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bedrooms        34786 non-null  float64
 1   bathrooms       30162 non-null  float64
 2   toilets         30855 non-null  float64
 3   parking spaces  20494 non-null  float64
 4   type            35554 non-null  object 
 5   town            35554 non-null  object 
 6   state           35554 non-null  object 
 7   price           35554 non-null  float64
dtypes: float64(5), object(3)
memory usage: 2.4+ MB


In [123]:
df.describe()

,bedrooms,bathrooms,toilets,parking spaces,price
count,3.478600e+04,30162.000000,30855.000000,20494.000000,3.555400e+04
mean,6.328854e+03,4.639580,5.416140,5.993022,7.953429e+08
std,1.179561e+06,4.174094,8.051241,28.409110,8.129543e+10
min,2.000000e+00,2.000000,2.000000,2.000000,4.000000e+05
25%,4.000000e+00,4.000000,5.000000,3.000000,6.200000e+07
50%,4.000000e+00,4.000000,5.000000,4.000000,1.000000e+08
75%,5.000000e+00,5.000000,6.000000,5.000000,1.850000e+08
max,2.200000e+08,500.000000,1300.000000,2000.000000,1.526615e+13


In [124]:
df.describe(include='O')

,type,town,state
count,35554,35554,35554
unique,49,194,26
top,detached duplex,Lekki,Lagos
freq,15027,16625,28998


In [125]:
def Missing(df):
    missing_numbers = df.isnull().sum().sort_values(ascending=False)
    missing_percent = round((df.isnull().sum() / len(df) * 100).sort_values(ascending=False),1)
    missing_values = pd.concat([missing_numbers,missing_percent],axis=1,keys = ['Missing Numbers','Missing Percentage'])
    return missing_values

In [126]:
Missing(df)

,Missing Numbers,Missing Percentage
parking spaces,15060,42.4
bathrooms,5392,15.2
toilets,4699,13.2
bedrooms,768,2.2
type,0,0.0
town,0,0.0
state,0,0.0
price,0,0.0


In [127]:
df = df.dropna(how = 'any')

In [128]:
df.isnull().sum()

bedrooms          0
bathrooms         0
toilets           0
parking spaces    0
type              0
town              0
state             0
price             0
dtype: int64

In [129]:
df.duplicated().sum()

8804

In [130]:
df = df.drop_duplicates()

In [131]:
df.duplicated().sum()

0

In [132]:
def DetectOutliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    Minimum = Q1 - 1.5 * IQR
    Maximum = Q3 + 1.5 * IQR
    return ((df < Minimum) | (df > Maximum)).sum()

CountOutliers = DetectOutliers(df) # Calling function here
CountOutliers 

bathrooms         1365
bedrooms          1091
parking spaces     682
price             1228
state                0
toilets            289
town                 0
type                 0
dtype: int64

In [133]:
def drop_outliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    l = df[~((df > upper) | (df < lower)).any(axis=1)]
    return l

In [134]:
df = drop_outliers(df)
df.shape

(8556, 8)

In [135]:
CountOutliers = DetectOutliers(df) # Calling function here
CountOutliers 

bathrooms           0
bedrooms            0
parking spaces    449
price             413
state               0
toilets           319
town                0
type                0
dtype: int64

In [136]:
df.to_csv('Cleaned Nigeria House Dataset.csv',index=False)